In [1]:
# imports
from support import *

import time
from datetime import datetime
from itertools import product
from math import erf, sqrt
import sys

# start the ib loop
util.startLoop()

# connect to the market set in variables.json
try:
    if not ib.isConnected():
        ib = get_connected(market, 'live')
except Exception as e:
    ib = get_connected(market, 'live')

# from json
a = assign_var(market) + assign_var('common')
for v in a:
    exec(v)

# reset the log
with open(logpath+'main.log', 'w'):
    pass # clear the run log

util.logToFile(logpath+'main.log')
    
jup_disp_adjust() # adjust jupyter's display

In [3]:
df = pd.read_pickle(fspath+'targets.pkl')

In [7]:
df.sort_values('RoM', ascending=False)

,symbol,optId,expiry,strike,right,optPrice,dte,undId,undPrice,lot,stDev,fall,rise,strikeRef,lo52,hi52,margin,comm,PoP,RoM,remq,qty,expPrice,expRom
97,IBULHSGFI,381436035,20191128,640.00,C,9.75,49,125585152,239.80,800,95.96,757.85,342.00,581.80,195.35,1377.50,47161.49,20.00,1.00,1.23,5,1,9.80,1.24
96,IBULHSGFI,381436041,20191128,660.00,C,8.95,49,125585152,239.80,800,95.96,757.85,342.00,581.80,195.35,1377.50,45571.49,20.00,1.00,1.17,5,1,9.00,1.17
95,IBULHSGFI,381436050,20191128,680.00,C,8.25,49,125585152,239.80,800,95.96,757.85,342.00,581.80,195.35,1377.50,44071.49,20.00,1.00,1.11,5,1,8.30,1.12
101,ICICIBANK,381441662,20191128,375.00,P,10.50,49,175596029,436.70,1375,15.24,60.90,90.20,375.80,259.25,451.95,106889.54,20.00,1.00,1.00,1,1,10.55,1.01
149,RBLBANK,381448961,20191128,470.00,C,9.90,49,246723708,309.60,1200,39.53,370.90,152.00,461.60,288.50,701.45,116007.39,20.00,1.00,0.76,2,1,13.05,1.01
148,RBLBANK,381448971,20191128,480.00,C,8.95,49,246723708,309.60,1200,39.53,370.90,152.00,461.60,288.50,701.45,112020.69,20.00,1.00,0.71,2,1,12.65,1.01
78,HDFCBANK,384794665,20191128,1440.00,C,20.15,49,382750910,1228.15,500,51.90,181.50,211.55,1439.70,955.88,1257.25,109853.40,20.00,1.00,0.68,1,1,29.65,1.00
147,RBLBANK,381448976,20191128,490.00,C,8.10,49,246723708,309.60,1200,39.53,370.90,152.00,461.60,288.50,701.45,108152.01,20.00,1.00,0.67,2,1,12.20,1.01
23,BANKNIFTY,378463183,20191010,27300.00,P,8.00,1,56994300,28786.00,20,360.75,1440.00,2685.50,27346.00,24421.00,31653.75,80477.10,20.00,1.00,0.63,1,1,12.70,1.06
157,SBIN,381436560,20191128,355.00,C,3.90,49,173418084,260.95,3000,22.54,120.40,80.90,341.85,233.20,372.40,140956.77,20.00,1.00,0.62,1,1,6.40,1.01


In [ ]:
ib = IB().connect('127.0.0.1', 3000, 0)

In [ ]:
df_ac = util.df(ib.accountSummary())

In [ ]:
NLV = float(df_ac[df_ac.tag.isin(['NetLiquidation'])].reset_index(drop=0).value[0])

In [ ]:
maintMargin = float(df_ac[df_ac.tag.isin(['InitMarginReq'])].reset_index(drop=0).value[0])

In [ ]:
df_ac.tag.unique()

In [ ]:
portf(ib)